In [ ]:
import sys
!{sys.executable} -m pip install kfp >/dev/null

In [ ]:
import os
import kfp

In [ ]:
dkube_training_op           = kfp.components.load_component_from_url("https://raw.githubusercontent.com/oneconvergence/dkube/master/components/training/component.yaml")
dkube_serving_op            = kfp.components.load_component_from_url("https://raw.githubusercontent.com/oneconvergence/dkube/master/components/serving/component.yaml")

In [ ]:
@kfp.dsl.pipeline(
    name='dkube-mnist-pl',
    description='sample mnist pipeline with dkube components'
)
def mnist_pipeline(token, program='training'):

    train       = dkube_training_op(token, '{"image":"ocdr/dkube-datascience-tf-cpu:v2.0.0"}',
                                    framework="tensorflow", version="2.0.0",
                                    program="training", run_script="python mnist/train.py",
                                    datasets='["training"]', outputs='["training"]',
                                    input_dataset_mounts='["/mnist"]',
                                    output_mounts='["/opt/dkube/output"]',
                                    envs='[{"EPOCHS": "2"}]')

    serving     = dkube_serving_op(token, train.outputs['artifact'], device='cpu', 
                                    serving_image='{"image":"ocdr/tensorflowserver:2.0.0"}',
                                    transformer_image='{"image":"ocdr/dkube-datascience-tf-cpu:v2.0.0"}',
                                    transformer_project="training",
                                    transformer_code='mnist/transformer.py').after(train)

In [ ]:
token  = os.getenv("DKUBE_USER_ACCESS_TOKEN")
client = kfp.Client(existing_token=token)
client.create_run_from_pipeline_func(mnist_pipeline, arguments={"token":token})

In [ ]:
#generate & upload pipeline (Optional)
import kfp.compiler as compiler
compiler.Compiler().compile(mnist_pipeline, "mnist-pipeline.zip")
client.upload_pipeline("mnist-pipeline.zip")